## Julia code for simulation

In [2]:
## necessary packages

using Distributions
using Distances
using LinearAlgebra
using SparseArrays
using IterativeSolvers
using ProgressMeter
using JLD2
using Random
using SpecialFunctions # Matern functions
using MLBase         # cross-validation 
using Convex         # compute stacking weights
using MosekTools     # compute stacking weights

In [3]:
include("./utils.jl")

sp_stacking_K_fold_V2 (generic function with 1 method)

In [4]:
# Set the parameters of the simulated data #
p = 2;      # No. covariates
β = [1.0 2.0]; #regression coeff
σ2 = 1.0; ϕ = 7.0; ν = 1.0; τ2 = 1.0; # hyperparmeters in matern

In [6]:
## Generate simulation data ##
Random.seed!(1);
N = 900;                     # No. all positions
N_ho = 100;                  # No. held out positions
ind_mod = 1:(N - N_ho);      # index of training observations
coords = rand(2, N);         # random location over unit square (2 by N)
X = vcat(fill(1.0, (1, N)), rand(Normal(), (1, N)));          # design matrix (p by N)
D = pairwise(Euclidean(), coords, dims = 2);                  # distance matrix
Cov = Symmetric(Maternlu.(UpperTriangular(D), 
        ν = ν, ϕ = ϕ, σ2 = σ2))                               # covariance matrix
z = rand(MvNormal(Cov), 1);                                   # latent process
y = (β * X)[1,:] + z[:,1] + sqrt(τ2) * rand(Normal(), N);     # response

In [7]:
using BenchmarkTools

In [8]:
## candidate values of hyperparameters for stacking ##
deltasq_grid = [0.1, 0.5, 1, 2];
phi_grid = [3, 9, 15, 21];
nu_grid = [0.5, 1, 1.5, 1.75];

In [9]:
## priors parameters ##
Priors = Dict("μβ" => fill(0.0, p), "inv_V_β" => Diagonal(ones(p) * 0.25), 
    "aσ" => 2.0, "bσ" => 2.0);

In [10]:
label = "LSE"; #stacking of means
K_fold = 10;

In [11]:
label = "LP";  #stacking of predictive densities
#J = 300;       # sample size for computing posterior expectation
K_fold = 10;

In [12]:
out = sp_stacking_K_fold(X, y, coords, deltasq_grid, phi_grid,
    nu_grid, Priors, K_fold = K_fold, seed = 432, label = label, J = 300);
[out[:grid_all][(out[:w].>0.01), :] out[:w][(out[:w].>0.01)]]

Computing initial pass... 100%|██████████████████████████████████████████████████| Time: 0:00:18



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



6×4 Matrix{Float64}:
  3.0  1.75  0.1  0.0343454
 21.0  1.0   0.5  0.0482017
 21.0  1.0   1.0  0.34265
  3.0  1.0   2.0  0.0854512
  3.0  1.5   2.0  0.0344804
 21.0  1.5   2.0  0.44069

In [13]:
out = sp_stacking_K_fold_V2(X, y, coords, deltasq_grid, phi_grid,
    nu_grid, Priors, K_fold = K_fold, seed = 432, label = label);
[out[:grid_all][(out[:w].>0.01), :] out[:w][(out[:w].>0.01)]]

6×4 Matrix{Float64}:
  3.0  1.75  0.1  0.0343454
 21.0  1.0   0.5  0.0482017
 21.0  1.0   1.0  0.34265
  3.0  1.0   2.0  0.0854512
  3.0  1.5   2.0  0.0344804
 21.0  1.5   2.0  0.44069

In [14]:
@benchmark sp_stacking_K_fold(X, y, coords, deltasq_grid, phi_grid,
    nu_grid, Priors, K_fold = K_fold, seed = 3214, label = label)

Computing initial pass... 100%|██████████████████████████████████████████████████| Time: 0:00:17
Computing initial pass... 100%|██████████████████████████████████████████████████| Time: 0:00:17
Computing initial pass... 100%|██████████████████████████████████████████████████| Time: 0:00:17
Computing initial pass... 100%|██████████████████████████████████████████████████| Time: 0:00:17


BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 17.610 s (0.74% GC) to evaluate,
 with a memory estimate of 3.18 GiB, over 65162978 allocations.

In [15]:
@benchmark sp_stacking_K_fold_V2(X, y, coords, deltasq_grid, phi_grid,
    nu_grid, Priors, K_fold = K_fold, seed = 3214, label = label)

BenchmarkTools.Trial: 2 samples with 1 evaluation.
 Range (min … max):  4.815 s …   4.917 s  ┊ GC (min … max): 2.11% … 1.50%
 Time  (median):     4.866 s              ┊ GC (median):    1.80%
 Time  (mean ± σ):   4.866 s ± 72.429 ms  ┊ GC (mean ± σ):  1.80% ± 0.43%

  █                                                       █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  4.81 s         Histogram: frequency by time        4.92 s <

 Memory estimate: 4.32 GiB, allocs estimate: 7542224.

In [15]:
@benchmark sp_stacking_K_fold(X, y, coords, deltasq_grid, phi_grid,
    nu_grid, Priors, K_fold = K_fold, seed = 3214, label = "LSE")

Computing initial pass... 100%|██████████████████████████████████████████████████| Time: 0:00:01
Computing initial pass... 100%|██████████████████████████████████████████████████| Time: 0:00:01
Computing initial pass... 100%|██████████████████████████████████████████████████| Time: 0:00:01
Computing initial pass... 100%|██████████████████████████████████████████████████| Time: 0:00:01
Computing initial pass... 100%|██████████████████████████████████████████████████| Time: 0:00:01
Computing initial pass... 100%|██████████████████████████████████████████████████| Time: 0:00:01
Computing initial pass... 100%|██████████████████████████████████████████████████| Time: 0:00:01
Computing initial pass... 100%|██████████████████████████████████████████████████| Time: 0:00:01


BenchmarkTools.Trial: 3 samples with 1 evaluation.
 Range (min … max):  1.948 s …   2.058 s  ┊ GC (min … max): 0.50% … 0.66%
 Time  (median):     1.949 s              ┊ GC (median):    0.50%
 Time  (mean ± σ):   1.985 s ± 63.181 ms  ┊ GC (mean ± σ):  0.55% ± 0.10%

  █                                                       ▁  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.95 s         Histogram: frequency by time        2.06 s <

 Memory estimate: 408.69 MiB, allocs estimate: 7160943.

In [16]:
@benchmark sp_stacking_K_fold_V2(X, y, coords, deltasq_grid, phi_grid,
    nu_grid, Priors, K_fold = K_fold, seed = 3214, label = "LSE")

BenchmarkTools.Trial: 9 samples with 1 evaluation.
 Range (min … max):  527.393 ms … 675.808 ms  ┊ GC (min … max): 1.79% … 2.55%
 Time  (median):     579.731 ms               ┊ GC (median):    2.22%
 Time  (mean ± σ):   577.403 ms ±  47.108 ms  ┊ GC (mean ± σ):  2.68% ± 0.74%

  ▁▁ ▁         ▁       █      ▁    ▁                          ▁  
  ██▁█▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁█▁▁▁▁▁▁█▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  527 ms           Histogram: frequency by time          676 ms <

 Memory estimate: 537.57 MiB, allocs estimate: 765937.

In [1]:
Threads.nthreads()

1

In [20]:
versioninfo()

Julia Version 1.9.3
Commit bed2cd540a1 (2023-08-24 14:43 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: macOS (arm64-apple-darwin22.4.0)
  CPU: 12 × Apple M2 Max
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-14.0.6 (ORCJIT, apple-m1)
  Threads: 9 on 8 virtual cores
Environment:
  LD_LIBRARY_PATH = /opt/homebrew/lib:/usr/local/lib:/usr/lib
  JULIA_NUM_THREADS = 8
